In [1]:
"""
Notebook version of the new clustering algorithm
"""

'\nNotebook version of the new clustering algorithm\n'

In [1]:
from yuval_module.paper_clusterer import PaperClusterer
from yuval_module.paper_source import PaperSource
import sys
import pandas as pd
import numpy as np
import s3_functions as s3func


%matplotlib inline

In [2]:
def load_dataset(set_name):
    ps=PaperSource()
    ps.load_dataset(set_name)
    return ps

In [3]:
def get_res_papers(ps,author_name):
    df=ps.get_dataset()
    return df[df['last_author_name']==author_name]

In [4]:
#ps=load_dataset('enriched_labeled')
ps=load_dataset('mini')

 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Expecting property name enclosed in double quotes: line 1 column 3 (char 2)


In [6]:
df=ps.get_dataset()
df.head()

,Unnamed: 0,Unnamed: 0.1,pmid,PI_IDS,last_author_name,authors,doi,journal_name,language,last_author,last_author_country,last_author_email,last_author_id,last_author_inst,mesh,mesh_major,pub_year,title
0,0,0,19810465,8352708,"Madabhushi, A",None,NaN,Medical physics,eng,NaN,Jersey,NaN,2517051.0,NaN,"['Humans', 'Linear Models', 'Male', 'Pilot Pro...","['Algorithms', 'Cluster Analysis', 'Diagnosis,...",2009.0,A hierarchical spectral clustering and nonline...
1,1,1,26167385,8352708,"Madabhushi, A",None,NaN,Journal of pathology informatics,eng,{'affiliation': 'Department of Biomedical Engi...,United States,NaN,NaN,"Rutgers, The State University of New Jersey",NaN,NaN,2015.0,Content-based image retrieval of digitized his...
2,2,2,26870745,8352708,"Madabhushi, A",None,NaN,"Journal of medical imaging (Bellingham, Wash.)",eng,{'affiliation': 'Case Western Reserve Universi...,United States,NaN,NaN,Case Western Reserve University,NaN,NaN,2015.0,Association of computerized texture features o...
3,3,3,25958195,8352708,"Madabhushi, A",None,10.1016/j.compmedimag.2015.04.002,Computerized medical imaging and graphics : th...,eng,{'affiliation': 'Department of Biomedical Engi...,United States,NaN,1576937.0,Case Western Reserve University,"['Coloring Agents', 'Female', 'Humans', 'Model...","['Algorithms', 'Breast/pathology', 'Color', 'I...",2015.0,Sparse Non-negative Matrix Factorization (SNMF...
4,4,4,25993029,8352708,"Madabhushi, A",None,NaN,PloS one,eng,{'affiliation': 'Department of Biomedical Engi...,United States,NaN,1576937.0,Case Western Reserve University,"['Algorithms', 'Datasets as Topic', 'Female', ...","['Breast Neoplasms/diagnosis', 'Diagnosis, Com...",2015.0,Predicting classifier performance with limited...


In [6]:
df=get_res_papers(ps,'Szeszko, PR')

In [7]:
df.head()

,Unnamed: 0,Unnamed: 0.1,pmid,PI_IDS,last_author_name,authors,doi,journal_name,language,last_author,last_author_country,last_author_email,last_author_id,last_author_inst,mesh,mesh_major,pub_year,title


In [8]:
def get_clusters(ps,author):
    author_df = get_res_papers(ps,author)
    paper_clusterer=PaperClusterer(eps=1.27)
    res_clusters, cluster_dfs=paper_clusterer.cluster_res(author_df)
    return cluster_dfs[0].sort_values(["cluster"])

In [9]:
def get_out_path(row):
    cur_name=row["last_author_name"]
    fields=cur_name.split(", ")
    return "/home/ubuntu/data/{}_{}.csv".format(fields[0], fields[1])

In [10]:
# def get_file_name(author_name):
#     author_name=author_name.replace(',',' ')
#     author_rebuilt='_'.join(author_name.split(' '))
#     return ''.join(["/home/ubuntu/data/", author_rebuilt, ".csv"])
# #%%
def main(author_list):
    print(author_list)
    
    for author in author_list:
        print(author)
        cluster_df=get_clusters(ps,author)
        # out_path=get_out_path(row)
        # cluster_df.to_csv(out_path, index=False, columns=["pmid", "cluster", "last_author_inst", "last_author_forename","pub_year","last_author_name"])
        
#         #pmid_df=pd.read_csv(list_path, dtype=str, usecols=["PMID"]).rename(columns={"PMID":"Highlight"})
#         #print(pmid_df.info())
#         #limit=min(limit, len(pmid_df))
#         #pmid_list=pmid_df.Highlight.tolist()[offset:offset+limit]
#         pmid_list=pmid_df.pmid.tolist()[:2]
#         paper_clusterer=PaperClusterer(eps=1.27)
#         res_clusters, cluster_dfs=paper_clusterer.infer_paper_author_data(pmid_list)
    
#     #res_clusters, cluster_dfs=paper_clusterer.infer_paper_author_data(pmid_list).sort_values(["rownum"])
#         return res_clusters, cluster_dfs
    #out_path="/home/ubuntu/data/rp_40000_authors.tsv"
    #out_path="/home/ubuntu/data/test_forenames_res.tsv"

    #res_clusters.to_csv(out_path, index=False, sep='\t')


In [11]:
author_list=['Madabhushi, A','Szeszko, PR']
main(author_list)

['Madabhushi, A', 'Szeszko, PR']
Madabhushi, A


TypeError: ('string indices must be integers', 'occurred at index 0')

In [28]:
with open("enriched_labeled_dataset_mini.csv", encoding="utf-8") as f:
    s3func.upload_to_s3(f,'enriched_labeled_dataset_mini.csv')

In [ ]:
print(cluster_dfs[0].cluster.value_counts())

In [ ]:
out_path="/home/ubuntu/data/hecht_jr.csv"
#out_path="/home/ubuntu/data/gottlieb_pa.csv"
cluster_dfs[0].sort_values(["cluster"]).to_csv(out_path, index=False, columns=["pmid", "cluster", "last_author_inst", "last_author_forename","pub_year","last_author_name"])

In [ ]:
# cur_clusters=cluster_dfs[0]
# print(cur_clusters.info())
# print(cur_clusters.cluster.value_counts())
# cur_clusters.to_csv("/home/ubuntu/data/friedman_n.csv", index=False)

In [ ]:
# cur_clusters=cluster_dfs[0][["pmid", "last_author_id", "last_author_inst", 
#                              "last_author_country", "last_author_name",
#                              "last_author_forename", "cluster",
#                             "pub_year", "last_author_affiliation"]].sort_values(["cluster"])
# print(cur_clusters)

In [ ]:
# cm = 'tab20b'
# cur_clusters=cluster_dfs[20][["cluster","pmid","last_author_inst",  "last_author_forename",
#                              "mesh_clean", "other_authors", "last_author_country", "last_author_name"]].reset_index()
# cur_clusters.sort_values('cluster').style.background_gradient(cmap=cm, subset=['cluster']).set_caption('Clustering output')

In [ ]:
def forename_delta(n1, n2):
        if ' ' in n1:
            n1=n1.split()[0]
        if ' ' in n2:
            n2=n2.split()[0]
        if n1==n2:
            if len(n1)>1:
                return 1.0
            else:
                return 0.5
        elif n1 in n2 or n2 in n1:
            return 0.75
        else:
            return 0.0

In [ ]:
forename_delta("Peter A","P A")